Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/responsible-ai/visualize-upload-loan-decision/rai-loan-decision.png)

# Assess Fairness, Explore Interpretability, and Mitigate Fairness Issues 

This notebook demonstrates how to use [InterpretML](interpret.ml), [Fairlearn](fairlearn.org), and the [Responsible AI Widget's](https://github.com/microsoft/responsible-ai-widgets/) Fairness and Interpretability dashboards to understand a model trained on the Census dataset. This dataset is a classification problem - given a range of data about 32,000 individuals, predict whether their annual income is above or below fifty thousand dollars per year.

For the purposes of this notebook, we shall treat this as a loan decision problem. We will pretend that the label indicates whether or not each individual repaid a loan in the past. We will use the data to train a predictor to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.

We will first train a fairness-unaware predictor, load its global and local explanations, and use the interpretability and fairness dashboards to demonstrate how this model leads to unfair decisions (under a specific notion of fairness called *demographic parity*). We then mitigate unfairness by applying the `GridSearch` algorithm from `Fairlearn` package.


## Install required packages

This notebook works with Fairlearn v0.4.6, and not later versions. If needed, please uncomment and run the following cell:

In [ ]:
# %pip install --upgrade fairlearn==0.4.6

After installing packages, you must close and reopen the notebook as well as restarting the kernel.

## Load and preprocess the dataset

For simplicity, we import the dataset from the `shap` package, which contains the data in a cleaned format. We start by importing the various modules we're going to use:

In [ ]:
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate

from sklearn import svm, neighbors, tree
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

import pandas as pd
import numpy as np

# SHAP Tabular Explainer
from interpret.ext.blackbox import KernelExplainer
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel

We can now load and inspect the data:

In [ ]:
dataset = fetch_openml(data_id=1590, as_frame=True)
X_raw, y = dataset['data'], dataset['target']
X_raw["race"].value_counts().to_dict()

We are going to treat the sex of each individual as a protected attribute (where 0 indicates female and 1 indicates male), and in this particular case we are going separate this attribute out and drop it from the main data. We then perform some standard data preprocessing steps to convert the data into a format suitable for the ML algorithms

In [ ]:
sensitive_features = X_raw[['sex','race']]

le = LabelEncoder()
y = le.fit_transform(y)

Finally, we split the data into training and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, sensitive_features_train, sensitive_features_test = \
    train_test_split(X_raw, y, sensitive_features,
                     test_size = 0.2, random_state=0, stratify=y)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
sensitive_features_train = sensitive_features_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

## Training a fairness-unaware predictor

To show the effect of `Fairlearn` we will first train a standard ML predictor that does not incorporate fairness. For speed of demonstration, we use a simple logistic regression estimator from `sklearn`:

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, make_column_selector(dtype_exclude="category")),
        ("cat", categorical_transformer, make_column_selector(dtype_include="category")),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegression(solver="liblinear", fit_intercept=True),
        ),
    ]
)

model.fit(X_train, y_train)

## Generate model explanations

In [ ]:
# Using SHAP KernelExplainer
# clf.steps[-1][1] returns the trained classification model
explainer = MimicExplainer(model.steps[-1][1], 
                           X_train,
                           LGBMExplainableModel,
                           features=X_raw.columns, 
                           classes=['Rejected', 'Approved'],
                           transformations=preprocessor)

### Generate global explanations
Explain overall model predictions (global explanation)

In [ ]:
# Explain the model based on a subset of 1000 rows
global_explanation = explainer.explain_global(X_test[:1000])

In [ ]:
global_explanation.get_feature_importance_dict()

### Generate local explanations
Explain local data points (individual instances)

In [ ]:
# You can pass a specific data point or a group of data points to the explain_local function
# E.g., Explain the first data point in the test set
instance_num = 1
local_explanation = explainer.explain_local(X_test[:instance_num])

In [ ]:
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = model.predict(X_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]

In [ ]:
print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

## Visualize model explanations
Load the interpretability visualization dashboard

In [ ]:
from raiwidgets import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, model, dataset=X_test[:1000], true_y=y_test[:1000])

We can load this predictor into the Fairness dashboard, and examine how it is unfair:

## Assess model fairness 
Load the fairness visualization dashboard

In [ ]:
from fairlearn.widget import FairlearnDashboard

y_pred = model.predict(X_test)

FairlearnDashboard(sensitive_features=sensitive_features_test,
                   y_true=y_test,
                   y_pred=y_pred)

Looking at the disparity in accuracy, we see that males have an error rate about three times greater than the females. More interesting is the disparity in opportunitiy - males are offered loans at three times the rate of females.

Despite the fact that we removed the feature from the training data, our predictor still discriminates based on sex. This demonstrates that simply ignoring a protected attribute when fitting a predictor rarely eliminates unfairness. There will generally be enough other features correlated with the removed attribute to lead to disparate impact.

## Mitigation with Fairlearn (GridSearch)

The `GridSearch` class in `Fairlearn` implements a simplified version of the exponentiated gradient reduction of [Agarwal et al. 2018](https://arxiv.org/abs/1803.02453). The user supplies a standard ML estimator, which is treated as a blackbox. `GridSearch` works by generating a sequence of relabellings and reweightings, and trains a predictor for each.

For this example, we specify demographic parity (on the protected attribute of sex) as the fairness metric. Demographic parity requires that individuals are offered the opportunity (are approved for a loan in this example) independent of membership in the protected class (i.e., females and males should be offered loans at the same rate). We are using this metric for the sake of simplicity; in general, the appropriate fairness metric will not be obvious.

In [ ]:
# Fairlearn is not yet fully compatible with Pipelines, so we have to pass the estimator only
X_train_prep = preprocessor.transform(X_train).toarray()
X_test_prep = preprocessor.transform(X_test).toarray()

sweep = GridSearch(LogisticRegression(solver="liblinear", fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=70)

Our algorithms provide `fit()` and `predict()` methods, so they behave in a similar manner to other ML packages in Python. We do however have to specify two extra arguments to `fit()` - the column of protected attribute labels, and also the number of predictors to generate in our sweep.

After `fit()` completes, we extract the full set of predictors from the `GridSearch` object.

In [ ]:
sweep.fit(X_train_prep, y_train,
          sensitive_features=sensitive_features_train.sex)

predictors = sweep._predictors

We could load these predictors into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the predictors which are dominated in the error-disparity space by others from the sweep (note that the disparity will only be calculated for the sensitive feature). In general, one might not want to do this, since there may be other considerations beyond the strict optimization of error and disparity (of the given protected attribute).

In [ ]:
from fairlearn.metrics import demographic_parity_difference

accuracies, disparities = [], []

for predictor in predictors:
    y_pred = predictor.predict(X_train_prep)
    # accuracy_metric_frame = MetricFrame(accuracy_score, y_train, predictor.predict(X_train_prep), sensitive_features=sensitive_features_train.sex)
    # selection_rate_metric_frame = MetricFrame(selection_rate, y_train, predictor.predict(X_train_prep), sensitive_features=sensitive_features_train.sex)
    accuracies.append(accuracy_score(y_train, y_pred))
    disparities.append(demographic_parity_difference(y_train,
                                                     y_pred,
                                                     sensitive_features=sensitive_features_train.sex))
    
all_results = pd.DataFrame({"predictor": predictors, "accuracy": accuracies, "disparity": disparities})

all_models_dict = {"unmitigated": model.steps[-1][1]}
dominant_models_dict = {"unmitigated": model.steps[-1][1]}
base_name_format = "grid_{0}"
row_id = 0
for row in all_results.itertuples():
    model_name = base_name_format.format(row_id)
    all_models_dict[model_name] = row.predictor
    accuracy_for_lower_or_eq_disparity = all_results["accuracy"][all_results["disparity"] <= row.disparity]
    if row.accuracy >= accuracy_for_lower_or_eq_disparity.max():
        dominant_models_dict[model_name] = row.predictor
    row_id = row_id + 1

We can construct predictions for all the models, and also for the dominant models:

In [ ]:
dashboard_all = {}
for name, predictor in all_models_dict.items():
    value = predictor.predict(X_test_prep)
    dashboard_all[name] = value
    
dominant_all = {}
for name, predictor in dominant_models_dict.items():
    dominant_all[name] = predictor.predict(X_test_prep)

FairlearnDashboard(sensitive_features=sensitive_features_test, 
                  y_true=y_test,
                  y_pred=dominant_all)

We can look at just the dominant models in the dashboard:

We see a Pareto front forming - the set of predictors which represent optimal tradeoffs between accuracy and disparity in predictions. In the ideal case, we would have a predictor at (1,0) - perfectly accurate and without any unfairness under demographic parity (with respect to the protected attribute "sex"). The Pareto front represents the closest we can come to this ideal based on our data and choice of estimator. Note the range of the axes - the disparity axis covers more values than the accuracy, so we can reduce disparity substantially for a small loss in accuracy.

By clicking on individual models on the plot, we can inspect their metrics for disparity and accuracy in greater detail. In a real example, we would then pick the model which represented the best trade-off between accuracy and disparity given the relevant business constraints.

# AzureML integration

We will now go through a brief example of the AzureML integration.

The required package can be installed via:

```
pip install azureml-contrib-fairness
pip install azureml-interpret
```

## Connect to workspace

Just like in the previous tutorials, we will need to connect to a [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py).

The following code will allow you to create a workspace if you don't already have one created. You must have an Azure subscription to create a workspace:

```python
from azureml.core import Workspace
ws = Workspace.create(name='myworkspace',
                      subscription_id='<azure-subscription-id>',
                      resource_group='myresourcegroup',
                      create_resource_group=True,
                      location='eastus2')
```

**If you are running this on a Notebook VM, you can import the existing workspace.**

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Registering models

The fairness dashboard is designed to integrate with registered models, so we need to do this for the models we want in the Studio portal. The assumption is that the names of the models specified in the dashboard dictionary correspond to the `id`s (i.e. `<name>:<version>` pairs) of registered models in the workspace.

Next, we register each of the models in the `dominant_all` dictionary into the workspace. For this, we have to save each model to a file, and then register that file:

In [ ]:
import joblib
import os
from azureml.core import Model, Experiment, Run

os.makedirs('models', exist_ok=True)
def register_model(name, model):
    print("Registering ", name)
    model_path = "models/{0}.pkl".format(name)
    joblib.dump(value=model, filename=model_path)
    registered_model = Model.register(model_path=model_path,
                                      model_name=name,
                                      workspace=ws)
    print("Registered ", registered_model.id)
    return registered_model.id

model_name_id_mapping = dict()
for name, model in dominant_all.items():
    m_id = register_model(name, model)
    model_name_id_mapping[name] = m_id

Now, produce new predictions dictionaries, with the updated names:

In [ ]:
dominant_all_ids = dict()
for name, y_pred in dominant_all.items():
    dominant_all_ids[model_name_id_mapping[name]] = y_pred

## Uploading a dashboard

We create a _dashboard dictionary_ using Fairlearn's `metrics` package. The `_create_group_metric_set` method has arguments similar to the Dashboard constructor, except that the sensitive features are passed as a dictionary (to ensure that names are available), and we must specify the type of prediction. Note that we use the `dashboard_registered` dictionary we just created:

In [ ]:
sf = { 'sex': sensitive_features_test.sex, 'race': sensitive_features_test.race }

from fairlearn.metrics._group_metric_set import _create_group_metric_set

dash_dict_all = _create_group_metric_set(y_true=y_test,
                                         predictions=dominant_all_ids,
                                         sensitive_features=sf,
                                         prediction_type='binary_classification')

Now, we import our `contrib` package which contains the routine to perform the upload:

In [ ]:
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

Now we can create an Experiment, then a Run, and upload our dashboard to it:

In [ ]:
exp = Experiment(ws, 'responsible-ai-loan-decision')
print(exp)

run = exp.start_logging()
try:
    dashboard_title = "Upload MultiAsset from Grid Search with Census Data Notebook"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict_all,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
finally:
    run.complete()

## Uploading  explanations




In [ ]:
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(run)
client.upload_model_explanation(global_explanation, comment = "census data global explanation")